In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import numpy as np
import pylab as plt

# Make sure we can find lassi-analysis
import sys
sys.path.append('/home/scratch/psalas/LASSI/lassi-analysis_v2')

from bad_scans import october_bad_scans as bad_scans
from utils.utils import makeGrid, zernikeResidualSurfaceError

In [18]:
def process(zFitDict, zindxs, zcoefs, eta_tot, verbose=False):
    """
    """
    
    eta_max = np.nanmax(eta_tot)
    
    # Extract information from the dictionaries and update wrong values.
    iz = np.zeros(len(zFitDict.keys()), dtype=int)
    izv = np.zeros(len(zFitDict.keys()), dtype=np.float)
    izvas = np.zeros(len(zFitDict.keys()), dtype=np.float)
    zobs = np.zeros(len(zFitDict.keys()), dtype=np.float)
    pdiff = np.zeros(len(zFitDict.keys()), dtype=np.float)
    scan = np.zeros(len(zFitDict.keys()), dtype=int)

    for i,k in enumerate(sorted(list(zFitDict.keys())[1:])):

        if k in bad_scans:
            continue

        scan[i] = k

        iz[i] = zFitDict[k]['input zernike']
        izv[i] = zFitDict[k]['input zernike value']
        izvas[i] = zFitDict[k]['active surface zernike value']
        zobs[i] = zFitDict[k]['recovered zernike'][iz[i]]*1e6
        pdiff[i] = zFitDict[k]['recovered zernike difference AS']
        
    mask = (iz == 0)
    iz = iz[~mask]
    izv = izv[~mask]
    izvas = izvas[~mask]
    zobs = zobs[~mask]
    scan = scan[~mask]
    
    # Compute statistics per coefficient, per polynomial.
    zobs_x = np.unique(izv)
    uiz = np.unique(iz)

    zobs_zi = np.zeros((len(zobs_x),len(uiz)), dtype=np.int)
    zobs_izv = np.zeros((len(zobs_x),len(uiz)), dtype=np.int)
    zobs_stds_vz = np.zeros((len(zobs_x),len(uiz)), dtype=np.float)
    zobs_means_vz = np.zeros((len(zobs_x),len(uiz)), dtype=np.float)
    zobs_pdiff_vz = np.zeros((len(zobs_x),len(uiz)), dtype=np.float)
    
    eta_res_all = []
    eta_res = np.zeros((len(zobs_x),len(uiz)), dtype=np.float)
    eta_res_p = np.zeros((len(zobs_x),len(uiz)), dtype=np.float)
    
    for i,izv_ in enumerate(zobs_x):
        for j,iz_ in enumerate(uiz):

            mask = (izv == izv_) & (iz == iz_)
            idxs = np.where(mask == True)[0]
            
            zobs_zi[i,j] = iz_
            zobs_izv[i,j] = np.nanmean(izvas[mask])
            
            # Index of Zernike polynomial in the aperture efficiency array.
            iz_idx = np.argmin(abs(zindxs - iz_))
            
            # Aperture efficiency given the residuals for each measurement.
            eta_res_ = np.zeros(len(idxs), dtype=np.float)
            
            for k,ix in enumerate(idxs):
                
                # Find the corresponding index for the coefficient.
                rzv = izvas[ix] - zobs[ix]
                idx = np.argmin(abs(rzv - zcoefs*1e6))
                
                eta_res_[k] = eta_tot[iz_idx,idx]
                eta_res_all.append(eta_tot[iz_idx,idx])
                #print(iz_, izv_, rzv, eta_res_[k], zobs[ix])
            
            eta_res[i,j] = np.mean(eta_res_)
            eta_res_p[i,j] = (eta_max - eta_res[i,j])/eta_max*100.
            
            zobs_stds_vz[i,j] = np.nanstd(zobs[mask])
            zobs_means_vz[i,j] = np.nanmean(zobs[mask])
            zobs_pdiff_vz[i,j] = np.nanmean((zobs[mask] - izvas[mask])/izvas[mask])
            
    return eta_res, eta_res_p, eta_res_all

In [26]:
# Grid parameters, in meters.
xmin = -50.
xmax = 50.
ymin = 0.1
ymax = 100.
dx   = (xmax - xmin)/512
dy   = (ymax - ymin)/512
# Surface error.
eps = 230e-6 # m
# Observing frequency.
nu = 115e9
lmbd = 3e8/nu
# Thermal deformations.
n_zern = 36
zindxs = [4,7,13,15]
zcoefs = np.arange(-500., 500., 10.)*1e-6 # m

In [27]:
# Make grid.
xx, yy, zz = makeGrid(xmin, xmax, dx, ymin, ymax, dy)
print(xx.shape)

(512, 512)


In [28]:
%%time

eps_tot = np.zeros((len(zindxs), len(zcoefs)), dtype=np.float)
eta_tot = np.zeros((len(zindxs), len(zcoefs)), dtype=np.float)

for i,iz in enumerate(zindxs):
    for j,zv in enumerate(zcoefs):
        z_coef = np.zeros(n_zern)
        z_coef[iz] = zv
        eps_tot[i,j], eta_tot[i,j], _ = zernikeResidualSurfaceError(xx, yy, z_coef, eps=eps, lmbd=lmbd, verbose=False)

CPU times: user 7min 38s, sys: 27.5 ms, total: 7min 38s
Wall time: 7min 38s


In [29]:
print(np.nanmax(eta_tot))

0.20804304085861067


In [7]:
print(np.nanmax(eta_tot))

0.20804304085861067


In [30]:
%%time
dictFiles = ['zFitDict_{}b.pickle'.format(i) for i in np.arange(1,45)]
#dictFiles = ['zFitDict_{}b.pickle'.format(i) for i in np.arange(15,16)]

eta_res = np.zeros((len(dictFiles),2), dtype=np.float)

for i,df in enumerate(dictFiles):
    zFitDict = pickle.load( open(df, "rb"), encoding='bytes')
    eta, etap, etall = process(zFitDict, zindxs, zcoefs, eta_tot, verbose=False)
    eta_res[i] = np.mean(eta), np.mean(etap)
    
    df_ = df.split('_')[-1].split('.')[0]
    
    try:
        print(df_, eta_res[i,1], np.max(etap), np.min(etall), zFitDict['header']['maskRadius'], zFitDict['header']['sigma'])
    except KeyError:
        print(df_, eta_res[i,1], np.max(etap), np.min(etall), zFitDict['header']['maskRadius'], 3)

#print(eta_res)

1b 8.481737595384127 32.49001782084947 0.11000209153504935 49 3
2b 6.88501378866417 19.412105747092944 0.14149306850580043 49 3.5
3b 5.452543836752968 12.72412675936988 0.1567300213983706 49 4
4b 5.419359254158886 8.92735768116435 0.1708975480821725 49 4.5
5b 5.242810294832496 9.3191758835003 0.1708975480821725 49 5
6b 5.5501905203374164 9.515557152119078 0.18158205394819843 49 5.5
7b 5.72478883852684 12.345183571994607 0.1708975480821725 49 6
8b 10.364601298464997 43.032634425727814 0.08030325787714375 48.5 3
9b 8.79527157901863 29.476437132384063 0.11000209153504935 48.5 3.5
10b 7.6743433441825415 23.19733186233785 0.1257432694787409 48.5 4
11b 6.495972094704085 16.386098058598638 0.14149306850580043 48.5 4.5
12b 6.288841815632835 15.994279856262688 0.14149306850580043 48.5 5
13b 6.78414717854915 16.386098058598638 0.14149306850580043 48.5 5.5
14b 6.219263279578357 16.386098058598638 0.14149306850580043 48.5 6
15b 4.678563353644687 7.749324943806493 0.1790485747326915 50 3
16b 5.6185

In [21]:
%%time
dictFiles = ['zFitDict_{}b.pickle'.format(i) for i in np.arange(1,45)]
#dictFiles = ['zFitDict_{}b.pickle'.format(i) for i in np.arange(15,16)]

eta_res = np.zeros((len(dictFiles),2), dtype=np.float)

for i,df in enumerate(dictFiles):
    zFitDict = pickle.load( open(df, "rb"), encoding='bytes')
    eta, etap, etall = process(zFitDict, zindxs, zcoefs, eta_tot, verbose=False)
    eta_res[i] = np.mean(eta), np.mean(etap)
    
    df_ = df.split('_')[-1].split('.')[0]
    
    try:
        print(df_, eta_res[i,1], np.max(etap), np.min(etall), zFitDict['header']['maskRadius'], zFitDict['header']['sigma'])
    except KeyError:
        print(df_, eta_res[i,1], np.max(etap), np.min(etall), zFitDict['header']['maskRadius'], 3)

#print(eta_res)

1b 8.887654252828744 34.31257900541464 0.10490361857687991 49 3
2b 7.2184239660639 20.61947118669336 0.13748873013842577 49 3.5
3b 5.6862317593649765 13.539356294972771 0.15345886521764618 49 4
4b 5.6409456061714 9.524577960999528 0.16841259762058575 49 4.5
5b 5.449299188853332 9.945452759140885 0.16841259762058575 49 5
6b 5.772626369976099 9.845709813279687 0.18067539402129673 49 5.5
7b 5.954275688240546 13.187387096176636 0.16841259762058575 49 6
8b 10.822607882623279 45.13964880763066 0.07480742047499032 48.5 3
9b 9.199694724728362 31.113124251519615 0.10490361857687991 48.5 3.5
10b 8.018342070173226 24.55451592003311 0.1211155566936525 48.5 4
11b 6.7703468988566184 17.377536849657595 0.13748873013842577 48.5 4.5
12b 6.544652145685204 16.95666205151623 0.13748873013842577 48.5 5
13b 7.0681048284021095 17.377536849657595 0.13748873013842577 48.5 5.5
14b 6.473069192640172 17.377536849657595 0.13748873013842577 48.5 6
15b 4.8743716030810855 7.926469555927085 0.178396801008113 50 3
16b 

In [18]:
best = np.argmin(eta_res[:,1])
print(dictFiles[best])
zFitDict = pickle.load( open(dictFiles[best], "rb"), encoding='bytes')
print(zFitDict['header'])

zFitDict_26b.pickle
{'scan0': 16, 'scanf': 125, 'nZern': 37, 'signal_registration': False, 'sigma_clip_diff': False, 'kernel_size': 1, 'do_replace_nans': False, 'guess': [60.0, 0.0, 0.0, -50.0, 0.0, 0.0], 'maskRadius': 49.5, 'fixed_reference': False, 'reference_scan': None, 'signal_type': b'SIGNAL', 'sigma': 4}
